In [1]:
import re

import pandas as pd

from issue import Issue
from tqdm import tqdm

In [1]:
DATA_FILE_PATH = "../vscode_issues_SA.csv.gzip" # "../teste.csv.gzip" # "../vscode_issues_SA.csv.gzip"

sample_dataset = pd.read_csv(DATA_FILE_PATH, compression='gzip', lineterminator='\n')

issues_dict_list = sample_dataset.to_dict('records')

issues = []
test_dataset = []

for issue_dict in tqdm(issues_dict_list):
	try:
		new_issue = Issue.from_dict(issue_dict)
		issues.append(issues)
	except Exception as e:
		pass
	

NameError: name 'pd' is not defined

In [3]:
print(len(training_dataset))

7512


In [6]:
def apply_steps_to_dataset(processing_funcs, dataset):
	"""
	Given a list of preocessing functions and a dataset (list of issues), 
	applies each function to the dataset in the order given.

	Each function must return the altered issue, unless they are
	supposed to be filtered out, in which case the function 
	must return None.
	"""
	
	new_issues = []

	for issue in dataset:
		for func in processing_funcs:
			issue = func(issue)
			if issue is None:
				break
		if issue is not None:
			new_issues.append(issue)
	
	print("New dataset has " + len(new_issues) + " issues")

	return new_issues

1000


### Pre-Processing Steps

In [ ]:
def filter_test_dataset(issue):
	"""
	Checks the issue id. If it is in the range of the test set (210000 < id <= 220000),
	return the issue. Otherwise, return None.
	"""

In [ ]:
def filter_main_training_dataset(issue):
	"""
	Checks the issue id. If it is in the range of the larger training set (id <= 210000),
	return the issue. Otherwise, return None.
	"""


In [ ]:
def filter_recent_issues_training_dataset(issue):
	"""
	Checks the issue id. If it is in the range of the training set which only contains
	recent issues (190000<= id <= 210000), return the issue. Otherwise, return None.
	"""


In [ ]:
def filter_basic_trainingset_requirements(issue):
	"""
	Checks if a given issue corresponds to the basic requirements for the
	training set are met These are vscode's issues that 
		(i) are closed; 
		(ii) have exactly one assignee;
	"""

In [ ]:
def filter_unfrequent_commiters(issue):
	"""
	Uses the code/technique from the `contribution_counter` notebook to filter out
	reviews from auhors with 
	"""

In [ ]:
def clean_issue_title(issue):
	"""
	Cleans the issue field of the given issue.
	"""
	new_title = issue.summary

	# Remove mention to other issues
	new_title = re.sub(r"\[?\s*[Ff]ollow up to #?[\d]+\s*\]?", "", new_title)

	# Remove monospacing markdown formatting
	new_title = re.sub(r"`([\s\S]*?)`", r"\1", new_title)

	return new_title


In [3]:
## BE CAREFUL IF ALTERING THESE CONSTANTS.
# They should be the same used for training the model in the training notbook (.ipynb)
CODE_BEGIN_SENTINEL = "<BoC>"
CODE_END_SENTINEL = "<EoC>"

def clean_issue_body(issue):
	"""
	Cleans the body field of the given issue.
	Additionally, envolves code fragments using the sentinel tokens
	from the training notebook.
	"""

	issue_body = issue.body

	if issue_body is None:
		return ""

	code_fragments = ""

	new_body = issue_body

	#  TODO: Dont just isolate all fragments, but preserve
	# their place in the issue body, surrounding them with 
	# the sentinel tokens

	## Note code fragments
	for match in re.findall(r"```([\s\S]*?)```", new_body):
		code_fragments += CODE_BEGIN_SENTINEL + match + CODE_END_SENTINEL + "\n"

	new_body = re.sub(r"```([\s\S]*?)```", "", new_body)

	# Remove headers
	new_body = re.sub("#+ ", "", new_body)

	# Removing emphasis might interfere with code fragments
	# Watchout if you want to fill the TODO, as these lines
	# will have to be taken care of

	new_body = re.sub(r"_([\s\S]*?)_", r"\1", new_body)
	new_body = re.sub(r"\*([\s\S]*?)\*", r"\1", new_body)
	new_body = re.sub(r"`(\s[\S]*?)`", r"\1", new_body)

	# Remove html tags
	new_body = re.sub(r"<[\s\S]*?>", r"", new_body)

	# Remove markdown links
	new_body = re.sub(r"\!?\[[\s\S]+\]\([\S]+\)", "", new_body)

	# Remove attachments
	new_body = re.sub(r"https?://[\S]+", "", new_body)

	new_body = re.sub(r"[\s]*\n+", "\n", new_body)

	new_body = code_fragments + new_body

	return new_body


## Pre-Processing the Issue Datasets

In [ ]:
clean_dataset = apply_steps_to_dataset([filter_basic_trainingset_requirements,\
										filter_unfrequent_commiters,\
										clean_issue_title,\
										clean_issue_body],issues)

main_training_dataset = apply_steps_to_dataset([filter_main_training_dataset],clean_dataset)
recent_issues_training_dataset = apply_steps_to_dataset([filter_recent_issues_training_dataset],clean_dataset)
test_dataset = apply_steps_to_dataset([filter_test_dataset],clean_dataset)

### Saving the results to a new file

In [ ]:
def save_issue_repo(new_path,issue_repo):
	issues_as_dicts = []
	print("Parsing collected issues.\nThis might take a few minutes")

	for issue in tqdm(issue_repo):
		# print(vars(issue))
	
		issues_as_dicts.append(issue.to_dict())

	issues_as_dataset = pd.DataFrame.from_dict(issues_as_dicts)
	issues_as_dataset.to_csv(new_path, compression='gzip', index=False)

In [ ]:
MAIN_TRAINING_DESTINATION_PATH = "../train_A.csv.gzip"
RECENT_TRAINING_DESTINATION_PATH = "../train_B.csv.gzip"
TEST_DESTINATION_PATH = "../test.csv.gzip"

save_issue_repo(MAIN_TRAINING_DESTINATION_PATH,main_training_dataset)
save_issue_repo(RECENT_TRAINING_DESTINATION_PATH,recent_issues_training_dataset)
save_issue_repo(TEST_DESTINATION_PATH,test_dataset)
